<a href="https://colab.research.google.com/github/KathyRoma/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)


Assignment- 
1- Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
2- Engineer at least two new features. (See below for explanation & ideas.)
3- Fit a linear regression model with at least two features.
4- Get the model's coefficients and intercept.
5- Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.
6- What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
**As always, commit your notebook to your fork of the GitHub repo.

In [313]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Removing outliers
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
# I don't need string columns for my models
df = df.drop(['description', 'display_address', 'street_address', 'interest_level'], axis = 1)

In [316]:
df.head()

,bathrooms,bedrooms,created,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,40.7145,-73.9425,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,40.7947,-73.9667,5465,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,40.7388,-74.0018,2850,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,40.7539,-73.9677,3275,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,40.8241,-73.9493,3350,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [317]:
# To divide test and train data, i need to convert this column to datetime format
import datetime as dt
df['created'] = df['created'].astype('datetime64[ns]')
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
latitude                       float64
longitude                      float64
price                            int64
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                          int64
swimming_pool                    int64
new_construction                 int64
terrace                          int64
exclusive                

In [0]:
df = df.sort_values(by=['created'])

In [0]:
## Feature 1:
#I create a feature with my own rent preferencies
df_me = df[['dishwasher', 'laundry_in_unit', 'fitness_center', 'high_speed_internet','terrace', 'cats_allowed']]

In [320]:
df['my_preferencies'] = df_me.sum(axis=1)
print(df.shape)
df.head()

(48817, 31)


,bathrooms,bedrooms,created,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,my_preferencies
5186,1.0,1,2016-04-01 22:12:41,40.7302,-73.9924,3195,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7945,1.0,0,2016-04-01 22:56:00,40.7576,-73.9677,2000,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6424,2.0,3,2016-04-01 22:57:15,40.7388,-73.9851,5850,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2
7719,1.0,1,2016-04-01 23:26:07,40.7939,-73.9738,2745,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2
1723,1.0,1,2016-04-02 00:48:13,40.7784,-73.9491,2400,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
## Feature 2:
df['core'] = df['bathrooms'] + df['bedrooms'] + df['elevator'] + df['terrace']
df['bed&bath'] = df['bedrooms'] + df['bathrooms']

In [322]:
df.head()

,bathrooms,bedrooms,created,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,my_preferencies,core,bed&bath
5186,1.0,1,2016-04-01 22:12:41,40.7302,-73.9924,3195,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
7945,1.0,0,2016-04-01 22:56:00,40.7576,-73.9677,2000,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1.0
6424,2.0,3,2016-04-01 22:57:15,40.7388,-73.9851,5850,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,6.0,5.0
7719,1.0,1,2016-04-01 23:26:07,40.7939,-73.9738,2745,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,3.0,2.0
1723,1.0,1,2016-04-02 00:48:13,40.7784,-73.9491,2400,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,2.0


In [0]:
# Now i split my data to train and test sets
train = df[df['created'] <= '2016-05-31']
test = df[df['created'] >= '2016-06-01']

In [324]:
train

,bathrooms,bedrooms,created,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,my_preferencies,core,bed&bath
5186,1.0,1,2016-04-01 22:12:41,40.7302,-73.9924,3195,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
7945,1.0,0,2016-04-01 22:56:00,40.7576,-73.9677,2000,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1.0
6424,2.0,3,2016-04-01 22:57:15,40.7388,-73.9851,5850,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,6.0,5.0
7719,1.0,1,2016-04-01 23:26:07,40.7939,-73.9738,2745,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,3.0,2.0
1723,1.0,1,2016-04-02 00:48:13,40.7784,-73.9491,2400,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,1.0,3,2016-05-30 16:24:15,40.7219,-73.9880,4400,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.0,4.0
26469,1.0,4,2016-05-30 18:57:50,40.6981,-73.9121,3500,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,5.0,5.0
37746,1.0,0,2016-05-30 20:05:21,40.7381,-73.9959,3050,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2.0,1.0
33882,1.0,2,2016-05-30 20:24:17,40.7147,-73.9373,2550,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,3.0


In [325]:
test

,bathrooms,bedrooms,created,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,my_preferencies,core,bed&bath
11474,1.0,1,2016-06-01 01:10:37,40.7071,-73.9475,2150,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0
19176,1.0,1,2016-06-01 01:11:06,40.6877,-73.9450,2795,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.0,2.0
16226,1.0,2,2016-06-01 01:11:12,40.7291,-73.9895,3495,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,3.0
37756,1.0,0,2016-06-01 01:11:52,40.7213,-73.9822,2100,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0
17946,1.0,2,2016-06-01 01:12:22,40.7986,-73.9372,2900,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19943,1.0,2,2016-06-29 17:47:34,40.7261,-74.0014,3045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0
16801,1.0,1,2016-06-29 17:56:12,40.7358,-73.9560,2648,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,3.0,2.0
32633,1.0,1,2016-06-29 18:14:48,40.7597,-73.9890,2650,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3.0,2.0
20560,1.0,3,2016-06-29 18:30:41,40.7156,-73.9540,3200,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.0,4.0


In [0]:
# I drop the time column in already filtered test and train
train = train.drop(['created'], axis=1)
test = test.drop(['created'], axis=1)

In [327]:
train

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,my_preferencies,core,bed&bath
5186,1.0,1,40.7302,-73.9924,3195,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2.0
7945,1.0,0,40.7576,-73.9677,2000,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,1.0
6424,2.0,3,40.7388,-73.9851,5850,1,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,6.0,5.0
7719,1.0,1,40.7939,-73.9738,2745,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,3.0,2.0
1723,1.0,1,40.7784,-73.9491,2400,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,1.0,3,40.7219,-73.9880,4400,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.0,4.0
26469,1.0,4,40.6981,-73.9121,3500,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,5.0,5.0
37746,1.0,0,40.7381,-73.9959,3050,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2.0,1.0
33882,1.0,2,40.7147,-73.9373,2550,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,3.0


In [0]:
# Never mind, this didn't work

# from scipy import stats
# train = train[(np.abs(stats.zscore(train)) < 3).all(axis=1)]

In [329]:
train.shape

(31839, 32)

In [0]:
# Make a model with two features I created
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
model = LinearRegression()
features = ['core', 'my_preferencies']
target = ['price']
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [331]:
# Baseline
y_train['price'].mean()

3575.737743019567

In [332]:
# Fit the model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [333]:
# Mean absolute error for training
y_pred_train = model.predict(X_train)
mean_absolute_error(y_train, y_pred_train)

849.5123116725213

In [334]:
# Mean absolute error for testing
y_pred_test = model.predict(X_test)
mean_absolute_error(y_test, y_pred_test)

860.8229118956574

In [335]:
# Coefficients
model.coef_[0]

array([730.21958142, 115.45216677])

In [336]:
# Train results for the model
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
print('RMSE for train:', rmse_train)
print('MAE for train:', mae_train)
print('R2 for train:', r2_train)

RMSE for train: 1293.9259485815219
MAE for train: 849.5123116725213
R2 for train: 0.4608547994353138


In [337]:
# Test results for the model
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
print('RMSE for test:', rmse_test)
print('MAE for test:', mae_test)
print('R2 for test:', r2_test)

RMSE for test: 1291.5740365908905
MAE for test: 860.8229118956574
R2 for test: 0.4632715325180905


In [0]:
# Let's try another model with different features
model2 = LinearRegression()
features = ['bed&bath', 'my_preferencies']
target = ['price']
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [339]:
model2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [342]:
# MAE of the second model is higher, so my first worked better
y_pred_train2 = model2.predict(X_train)
mean_absolute_error(y_train, y_pred_train)

860.0005847886358

In [341]:
model2.coef_[0]

array([764.750069 , 267.5875566])

In [343]:
# Train results for model2
mse_train2 = mean_squared_error(y_train, y_pred_train2)
rmse_train2 = np.sqrt(mse_train)
mae_train2 = mean_absolute_error(y_train, y_pred_train2)
r2_train2 = r2_score(y_train, y_pred_train2)
print('RMSE for train:', rmse_train2)
print('MAE for train:', mae_train2)
print('R2 for train:', r2_train2)

RMSE for train: 1293.9259485815219
MAE for train: 860.0005847886358
R2 for train: 0.45854720927258363


In [345]:
y_pred_test2 = model2.predict(X_test)
mean_absolute_error(y_test, y_pred_test2)

871.1754393566712

In [346]:
# Test results for model2
mse_test2 = mean_squared_error(y_test, y_pred_test2)
rmse_test2 = np.sqrt(mse_test)
mae_test2 = mean_absolute_error(y_test, y_pred_test2)
r2_test2 = r2_score(y_test, y_pred_test2)
print('RMSE for test:', rmse_test2)
print('MAE for test:', mae_test2)
print('R2 for test:', r2_test2)

RMSE for test: 1291.5740365908905
MAE for test: 871.1754393566712
R2 for test: 0.4593194738127827
